# **Machine Learning Group Project:** Recommender System Preparation

The purpose of this notebook is to extract content-based similarities across different products on the basis of the plethora of of textual data contained in descriptive columns such as _tags_, _genres_, _description_, or _steamspy_tags_. These findings can later be utilized when building other models.

In [1]:
# General
import requests
import string
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

# Visualization
import seaborn as sns
from kneed import KneeLocator
import matplotlib.pyplot as plt

# Sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Parallelization
from joblib import Parallel, delayed

## Content-Based Similarity

### Text Data DataFrame

First we create a DataFrame that contains all instances of descriptive textual data for the products. There is a single entry for each game. We run an inner merge because this is the same kind of merge done between these datasets for the final_df in [a_csv_creation.ipynb](a_csv_creation.ipynb). Consequently, this ensures the data we are working with is compatible with other work later on.

In [ ]:
# Import the final DataFrame
textual_df = pd.read_csv("data/final_df.csv", 
                       usecols = ["app_id",
                                  "categories", "genres",
                                  "steamspy_tags", "description",
                                  "tags"])

In [ ]:
textual_df.drop_duplicates(subset=['app_id'], inplace=True)
textual_df.reset_index(drop = True, inplace = True)

textual_df.head(3)

### Data Preparation

#### Text Homogenization

The first step undertaken is homgenizing the text across the different columns. We do the following:
- Make all words fully lowercase
- Transform the _tags_ column from a list to a string
- Remove separators in the strings of columns such as _categories_, _steamspy_tags_, etc.
- Remove any other punctuation
- Homogenize words with spelling discrepancies (e.g. multiplayer & multi-player)

Note that we do not remove duplicates (e.g. a game being described as "Action" under _genres_ and _steamspy_tags_) because we believe if a word is utilized by more than one source, it should have a stronger weight when vectorizing. 

In [ ]:
# Turn to String
textual_df["tags"] = textual_df["tags"].astype(str)

# Remove all punctuation and make everything lowercase
textual_df["tags"] = textual_df["tags"].str.translate(str.maketrans('', '', string.punctuation)).str.lower()

In [ ]:
# Use vectorized operations to remove separators from list columns and turn all to lowercase
for col in ["categories", "genres", "steamspy_tags", "tags"]:
    textual_df[col] = textual_df[col].str.replace(';', ' ').str.lower()

    textual_df[col] = textual_df[col].str.replace("multi-player", "multiplayer")\
        .str.replace("free to play", "freetoplay").str.replace("single-player", "singleplayer")\
        .str.replace("post-apocalyptic", "postapocalyptic").str.replace("scifi", "sci-fi")\
        .str.replace("anti cheat", "anticheat").str.replace("early access", "earlyaccess")\
        .str.replace("shared/split screen", "shared/splitscreen").str.replace(" &", "")\
        .str.replace("sexual content", "sexual").str.replace(" (require", "").str.replace(" hl2)", "")\
        .str.replace("massively multiplayer", "massive-multiplayer")\
        .str.replace("in-app purchases", "inapppurcases").str.replace("perma death", "permadeath")\
        .str.replace("third person", "thirdperson").str.replace("multiple endings","multiending")\
        .str.replace("1990's", "1990s").str.replace("25d", "2.5d").str.replace("shoot 'em", "shootem")\
        .str.replace("co-op", "coop")

In [ ]:
# Define punctuation and numbers
punc = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
num = '0123456789'

# Remove punctuation, numbers, and turn description to lowercase 
textual_df["description"] = textual_df["description"].str.replace(f'[{punc+num}]', '', regex=True)\
    .str.lower().str.replace("multi-player", "multiplayer")

#### Stopword Removal

Although columns like _categories_, _genres_, _steamspy_tags_, and _tags_ do not suffer from this issue, the _description_ column contains stopwords which could be detrimental to our analysis. The stopword list is obtained from a separate [GitHub project](https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt).

In [ ]:
# Obtain stopwords
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = set(stopwords_list.decode().splitlines())

# Remove Stopwords
textual_df['description'] = textual_df['description']\
.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

#### Full Text Column Creation

Finally, we create a new column which contains all the textual information we have on a game.   

In [ ]:
textual_df['full_text'] = textual_df['categories'] + ' ' + textual_df['genres'] + ' ' + textual_df['steamspy_tags'] + ' ' + textual_df['description'] + ' ' + textual_df['tags']

### Text Data Exploration

We dedicate a section to exploring text data and truly understanding the breadth of data and labels which are available on textual_df.

**Number of Distinct Labels:** All four labelling columns feature different tag systems with wholly different labels (and variety thereof). Here we check how these vary in terms of numbers. While _categories_ and _genres_ have a relatively restrained count of unique labels (41 and 25 respectively), _tags_ and _steamspy_tags_ have much greater breadth (with 479 and 373 respectively). 

In [ ]:
for i in ["categories", "genres", "steamspy_tags", "tags"]:
    split_count = len(textual_df[i].str.split(' ', expand=True).stack().unique())
    print(f"The \033[1m'{i}' column\033[0m features a total of \033[1m{split_count} distinct labels\033[0m.")

**Average Number of Labels per Game:** While there is a great difference in the number of unique labels, the average number of labels assigned to a single game is on average quite stable. In fact, _genres_, _steamspy_tags_, and _tags_ all approximately have the same number of labels per row (to two decimal figures). Despite having the second lowest number of different labels, _categories_ has the highest number of labels per row: 8.12.

In [ ]:
for i in ["categories", "genres", "steamspy_tags", "tags"]:
    
    split = [len(j) for j in textual_df[i].str.split(' ', expand=True).stack()]
    split_mean = sum(split) / len(split)
    
    print("The entries in the\033[1m", f"'{i}'", "column\033[0m have an approximate average of\033[1m", 
          round(split_mean,2), "labels per row\033[0m.") 

**Most Common Labels:** AAAAA WE INVESTIGATE

In [ ]:
for i in ["categories", "genres", "steamspy_tags", "tags"]:
    
    split_count = textual_df[i].str.split(' ', expand=True).stack().value_counts()
    
    print("The most frequently recurring tags (along with their occurrence) in the", f"'{i}'",
          "column:", f"\n{split_count.nlargest(10)}\n")

**Tag Intersection:** Finally, we check if there are certain tags which exist in multiple columns (e.g. "action" being an entry in the _genres_, but also in _tags_. We find that there are AAAAAAAAAA

In [ ]:
# Get the unique values in each column
categories = set(textual_df['categories'].str.split(' ').explode().str.strip())
tags = set(textual_df['tags'].str.split(' ').explode().str.strip())
steamspy_tags = set(textual_df['steamspy_tags'].str.split(' ').explode().str.strip())
genre = set(textual_df['genres'].str.split(' ').explode().str.strip())

# Find the intersection of the sets to get the values that exist in all four columns
or_intersection = categories | tags | steamspy_tags | genre
and_intersection = categories & tags & steamspy_tags & genre

# Print the number of unique values that exist in all four columns
print("The number of labels present in at least two columns is:", len(or_intersection))
print("The number of labels present in all four columns is:", len(and_intersection))

#### Textual Data Vectorization

First we create a vector containing information concerning the textual descriptions of the games. To this end, TF vectorization through TfidfVectorizer. Note that we purposely decide not to make use of idf. We do this on the basis of the notion that that if a certain label is common in a game's metadata across multiple sources which tag the games, it is clearly more appropriate to describe the game than the others. We remove IDF because of the imbalance in labels across sources. Labels which are shared across sources are likely to be the more common and recognizable ones utilized on the platform. They would be the ones to be penalized, in favor of very niche terms in the much larger  By maintaining IDF, it is likely that these more descriptive terms would be lost in favor of more niche terminology.

In [ ]:
# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = "english",
                            strip_accents = "ascii",
                            use_idf = False)

# Fit and transform full text column
fulltext_vector = vectorizer.fit_transform(textual_df['full_text'])
fulltext_array = fulltext_vector.toarray()

# Get the feature names
feature_names = vectorizer.vocabulary_.keys()

# Create DataFrame
vector_df = pd.DataFrame(fulltext_array, columns = feature_names,
                        index = textual_df["app_id"])
vector_df.head(3)

#### Optimal Component Selection: Scree Test

The ultimate objective is to utilize Truncated Singular Value Decomposition to reduce the textual data's dimensionality and create a latent matrix. In order to do that, we conduct a Scree Test to determine the optimal number of components to be used. We decide that we want at least 90% of variance being explained by the decomposition. We find the minimum number of components necessary to achieve this and use the elbow method to identify the optimal number of components under the constraint that it explains at least 90% of data.

In order to identify the knee point, we use the [_kneed_](https://pypi.org/project/kneed/) library, which is based on the [_kneedle algorithm_](https://raghavan.usc.edu//papers/kneedle-simplex11.pdf) developed by Ville Satopaa, Jeannie Albrecht, David Irwin, and Barath Raghavan. 

**Note:** Fitting the svd model requires a lot of time. It is advised to only run the block of code below only once. 

In [ ]:
# Create Instance of SVD with 300 Components
svd = TruncatedSVD(n_components=300, random_state = 70)

# Fit to the Matrix
scree_matrix = svd.fit(vector_df)

In [ ]:
# Find Cumulative Explained Variance Percentage
cum_variance = np.cumsum(scree_matrix.explained_variance_ratio_)

# Identify First Instance of Expl. Variance Above 90%
ninety_threshold = np.argmax(cum_variance >= 0.9) 

print(f"The number of features required to explain at least 90% of variance is {ninety_threshold + 1}")

In [ ]:
# Obtain Explained Variance
expl_variance = scree_matrix.explained_variance_

# Find Elbow Point
kneedle = KneeLocator(np.arange(ninety_threshold + 1, len(expl_variance)+ 1), 
                      expl_variance[ninety_threshold:], curve='convex', 
                      direction='decreasing')

elbow_location = kneedle.elbow

# Plot Elbow
sns.set_theme()

plt.plot(np.arange(1, len(expl_variance)+1),
         expl_variance[0:],
         label = "Explained Variance")

# Add Title and Axis Labels
plt.title("Scree Test for SVD Analysis")
plt.ylabel("Explained Variance")
plt.xlabel("Number of Components")
plt.legend()

# Add Annotation and Point to Show Location of 90%
plt.annotate(f"90% of Variance Explained at {ninety_threshold + 1} Components", 
             xy=(ninety_threshold + 1, expl_variance[ninety_threshold]),
             xytext=(40, 0.010),
             arrowprops=dict(arrowstyle="-",
                             color = "blue"), fontsize=10)

plt.plot(ninety_threshold + 1, expl_variance[ninety_threshold], "bo",
         label = "Elbow Point")

# Add Annotation and Point to Show Location of Elbow
plt.annotate(f"Post-90% Elbow at {elbow_location} Components", 
             xy=(elbow_location, expl_variance[elbow_location - 1]),
             xytext=(200, 0.015),
             arrowprops=dict(arrowstyle="-",
                             color = "red"), fontsize=10)

plt.plot(elbow_location, expl_variance[elbow_location - 1], "ro",
         label = "Elbow Point")

#### Optimized SVD

Plotting the explained variance against the number of SVD components, we find that 90% of variance is in fact explained far past the plot's actual elbow. Finding the "post-90%" elbow and identifying it on the plot shows that it is located on the tail of the plot. It also consists of only 4 components more than the 90% variance point. Consequently, we decide to stick to our minimum variance point of 156 components with at least 90% variance as opposed to the 160 of the post-90% elbow. We believe the returns to making the model more complex are not sufficient.

We run the truncated SVD with 156 components.

In [ ]:
# Create Instance of SVD with 160 Components
svd_opt = TruncatedSVD(n_components=160, random_state = 70)

# Fit to the Matrix
lat_matrix = svd_opt.fit_transform(vector_df)

In [ ]:
# Turn Into DataFrame
lat_df = pd.DataFrame(lat_matrix)

# Add Movie Labels
titles_df = pd.concat([pd.Series(vector_df.index), lat_df], axis = 1)

# Cosine Similarity
cos_similarity = cosine_similarity(titles_df.iloc[:, 1:])

content_similarity_df = pd.DataFrame(cos_similarity, index = titles_df["app_id"], 
             columns = titles_df["app_id"])

In [ ]:
content_similarity_df.head()

In [ ]:
# Save df to file
content_similarity_df.to_csv('data/content_base.csv', index = True)

In [ ]:
# Delete Unnecessary DataFrames
del textual_df
del svd
del lat_df

## User Similarity

Now we create a DataFrame that maps the similarity across users. We do this based on a latent matrix mapping their ratings on individual Steam games. More specifically, we look to find the similarity between all those users which provide a single review and those which provide more than one. This will be useful later for the creation of a similarity column.

### Import DataFrame

We import the dataframe with the y, app_id, and user_id. We replace 0s with -1s in order to clearly denote a negative review. We also create two DataFrames, one with users with who have only one review, and one with users who have more than one.

In [2]:
# Import the user DataFrame
user_df = pd.read_csv("data/final_df.csv", 
                       usecols = ["app_id", "user_id",
                                 "y"])

# Turn Booleans to Integers
user_df["y"] = user_df["y"].astype(int).replace(0, -1)

In [3]:
# Create DF for users with only 1 review and more than 1 
duplicated_user_ids = user_df[user_df.duplicated(subset=["user_id"], keep=False)]["user_id"].unique()
multi_review = user_df[user_df["user_id"].isin(duplicated_user_ids)].reset_index(drop=True)

duplicated_user_ids = user_df[user_df.duplicated(subset=["user_id"], keep=False)]["user_id"].unique()
single_review = user_df[user_df["user_id"].isin(set(user_df["user_id"]) - set(duplicated_user_ids))].reset_index(drop=True)

# print the resulting dataframe
if (len(user_df) == (len(multi_review) + len(single_review))):
    print("No Datapoints Lost")

No Datapoints Lost


### Create Pivot Table and Train Model

In [4]:
# Create Pivot Table
rec_pivot = user_df.pivot_table(index = "user_id", columns = "app_id",
                   values = "y")

# Fill NaNs
rec_pivot = rec_pivot.fillna(0)

# Convert DataFrame to Sparse Matrix
sparse_recs = csr_matrix(rec_pivot.values)

In [ ]:
# Create Instance of SVD with 250 Components
user_svd = TruncatedSVD(n_components=250, random_state = 70)

# Fit to the Matrix
user_svd_fit = user_svd.fit(sparse_recs)

In [ ]:
# Find Cumulative Explained Variance Percentage
user_cum_variance = np.cumsum(user_svd_fit.explained_variance_ratio_)

# Identify First Instance of Expl. Variance Above 90%
user_ninety_threshold = np.argmax(user_cum_variance >= 0.9) 

print(f"The number of features required to explain at least 90% of variance is {user_ninety_threshold + 1}")

In [ ]:
# Obtain Explained Variance
user_expl_variance = user_svd_fit.explained_variance_

# Find Elbow Point
user_kneedle = KneeLocator(np.arange(user_ninety_threshold + 1, len(user_expl_variance)+ 1), 
                      expl_variance[user_ninety_threshold:], curve='convex', 
                      direction='decreasing')

user_elbow_location = user_kneedle.elbow

# Plot Elbow
sns.set_theme()

plt.plot(np.arange(1, len(user_expl_variance)+1),
         user_expl_variance[0:],
         label = "Explained Variance")

# Add Title and Axis Labels
plt.title("Scree Test for SVD Analysis")
plt.ylabel("Explained Variance")
plt.xlabel("Number of Components")
plt.legend()

# Add Annotation and Point to Show Location of 90%
plt.annotate(f"90% of Variance Explained at {user_ninety_threshold + 1} Components", 
             xy=(user_ninety_threshold + 1, user_expl_variance[user_ninety_threshold]),
             xytext=(40, 0.010),
             arrowprops=dict(arrowstyle="-",
                             color = "blue"), fontsize=10)

plt.plot(user_ninety_threshold + 1, user_expl_variance[user_ninety_threshold], "bo",
         label = "Elbow Point")

# Add Annotation and Point to Show Location of Elbow
plt.annotate(f"Post-90% Elbow at {user_elbow_location} Components", 
             xy=(user_elbow_location, user_expl_variance[user_elbow_location - 1]),
             xytext=(200, 0.015),
             arrowprops=dict(arrowstyle="-",
                             color = "red"), fontsize=10)

plt.plot(user_elbow_location, user_expl_variance[user_elbow_location - 1], "ro",
         label = "Elbow Point")

### Create Latent Matrix and Find Cosine Similarity

In [ ]:
# Transform Data into Latent Matrix
user_latent = pd.DataFrame(user_svd_fit.transform(sparse_recs),
                          index = rec_pivot.index)
user_latent.head()

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [ ]:
# Find cosine similarity between single-review and multi-review
row_users = single_review["user_id"].unique()
col_users = multi_review["user_id"].unique()

# Only take the optimal number of dimensions
single_review_latent = transformed_recs[row_users, :n]
user_lat_matrix = transformed_recs[col_users, :n]

# Compute the Cosine Similarity Matrix Between Rows of the Latent Matrix for the Selected User IDs
user_similarity_matrix = cosine_similarity(single_review_latent, user_lat_matrix)

# Create a DataFrame with the similarity matrix, indexed by row and column user IDs
user_similarity_df = pd.DataFrame(similarity_matrix, index=row_users, columns=col_users,
                                 index = row_users)

In [70]:
user_similarity_df.head()

In [ ]:
del user_lat_matrix
del single_review_latent
del row_users
del col_users
del user_latent

## Feature Creation

Finally, we create a brand new feature in our final_df. 

In [202]:
content_similarity_df = pd.read_csv("data/content_base.csv", index_col = "app_id")

In [223]:
content_similarity_df

app_id,304390,306130,238960,730,255710,289070,431960,635260,392160,570,...,489520,555950,394510,410320,285920,17470,403640,314160,704850,485510
app_id,,,,,,,,,,,,,,,,,,,,,
304390,1.000000,0.394583,0.652058,0.480446,0.204640,0.360051,0.138724,0.433073,0.259101,0.490285,...,0.530043,0.301991,0.514234,0.519567,0.406684,0.331687,0.332657,0.335709,0.301892,0.601789
306130,0.394583,1.000000,0.619674,0.269352,0.157970,0.178135,0.111520,0.387001,0.246213,0.332852,...,0.304212,0.212217,0.138190,0.220257,0.471072,0.198846,0.326764,0.343884,0.307128,0.509522
238960,0.652058,0.619674,1.000000,0.443977,0.166088,0.248202,0.171930,0.434388,0.245030,0.554080,...,0.467581,0.314532,0.415626,0.431764,0.416878,0.274098,0.311742,0.285849,0.275666,0.731631
730,0.480446,0.269352,0.443977,1.000000,0.277238,0.355281,0.245533,0.410403,0.312661,0.548046,...,0.445600,0.275654,0.472461,0.507455,0.364353,0.190225,0.323074,0.290928,0.219978,0.474242
255710,0.204640,0.157970,0.166088,0.277238,1.000000,0.440692,0.280810,0.287209,0.519933,0.248412,...,0.254262,0.320823,0.127167,0.230673,0.312222,0.096302,0.298973,0.288785,0.371059,0.302191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17470,0.331687,0.198846,0.274098,0.190225,0.096302,0.069726,0.103804,0.121625,0.292479,0.125171,...,0.092875,0.476745,0.271316,0.389122,0.298862,1.000000,0.311108,0.174169,0.273764,0.297453
403640,0.332657,0.326764,0.311742,0.323074,0.298973,0.184740,0.244294,0.226281,0.371855,0.219213,...,0.187007,0.424635,0.161784,0.373376,0.358941,0.311108,1.000000,0.201534,0.387046,0.463447
314160,0.335709,0.343884,0.285849,0.290928,0.288785,0.315979,0.097088,0.465208,0.406012,0.240811,...,0.272634,0.169885,0.232270,0.270962,0.386592,0.174169,0.201534,1.000000,0.443385,0.286356


In [ ]:
# Similarity Generator
class ContentSimilarityGenerator(BaseEstimator, TransformerMixin):
    def __init__(self, users, similarity):
        self.users = users
        self.similarity = similarity
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        similarities = []
        for i in range(0, len(X)):
            user = X.at[i, 'user_id']
            game = X.at[i, 'app_id']
            sim = self.AverageSimilarityCalculator(user, game)
            similarities.append(sim)
        return similarities
    
    def AverageSimilarityCalculator(self, user, game):
        user_ratings = self.users[self.users["user_id"] == user]
        other_games = user_ratings[user_ratings['app_id'] != game]
        other_games_titles = other_games['app_id']
        filtered_game_similarity_df = self.similarity.loc[game, other_games_titles] * other_games.set_index('app_id')["y"]
        return filtered_game_similarity_df.mean()



In [48]:
# Find Average Similarity
def AverageSimilarityCalculator(user, game):
    
    user_ratings = multi_review[multi_review["user_id"] == user]
    other_games = user_ratings[user_ratings['app_id'] != game]
    
    other_games_titles = other_games['app_id']
    filtered_game_similarity_df = content_similarity_df.loc[game, other_games_titles] * other_games.set_index('app_id')["y"]
    
    return filtered_game_similarity_df.mean()

# Create Compiler of Avg Sim
def SimilarityGenerator(users = multi_review, 
                     similarity = content_similarity_df):
    
    similarities = []
    for i in range(0, len(users)):
        
        sim = AverageSimilarityCalculator(users.at[i, 'user_id'],
                      users.at[i, 'app_id'])
        
        similarities.append(sim)
    
    return similarities

In [71]:
# Find Most Similary User
def UserPairer(user,
              user_similarity = user_similarity_df):
    
    return user_similarity_df.loc[user].idxmax()
    
# Create Similarity Imputer
def SimilarityImputer(users = single_review):
    
    sim_list = []
    for i in range(0, len(users)):
        similar_user = UserPairer(users.at[i, "user_id"])
        
        sim = AverageSimilarityCalculator(similar_user,
                                   users.at[i, "app_id"])
        
        sim_list.append(sim_list)
    
    return sim_list

In [49]:
# Call Similarity Generator
multi_similarities = SimilarityGenerator()

In [54]:
# Create New Column with Similarities for multi_review
multi_review["similarities"] = multi_similarities

multi_review

,y,app_id,user_id,similarities
0,1,306130,17622,0.500506
1,1,255710,125959,0.279530
2,1,431960,72938,0.068579
3,1,635260,29571,-0.625352
4,-1,392160,144843,-0.567523
...,...,...,...,...
3489009,1,225540,2894100,0.258590
3489010,1,225540,4252049,0.561824
3489011,1,225540,2843922,0.569846
3489012,1,225540,2173819,0.466156


In [55]:
# Save df to file
multi_review.to_csv('data/multi_review.csv', index = False)

In [ ]:
imputed_similarities = SimilarityImputer()